In [ ]:
import pandas as pd
import re

df = pd.read_csv("data/elon_tweets/2021.csv")
df = df[['date', 'tweet']]
df.drop(df.tail(1).index, inplace=True)
df = df[::-1].reset_index(drop=True)

In [11]:
for i in range(len(df)):
    if "http" in df["tweet"][i]:
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|[?:%[0-9a-fA-F][0-9a-fA-F])+', df["tweet"][i])

        for url in urls:
            df["tweet"][i] = df["tweet"][i].replace(url, '{URL}')

In [ ]:
df.to_csv("data/elon_tweets/ElonMuskTweetsPreProcessed.csv", index=False)